## 2.3 통계 기반 기법
자연어 처리를 위해서 말뭉치(corpus)를 이용할 것이다. 말뭉치는 대량의 텍스트 데이터이다.

### 2.3.1 파이썬으로 말뭉치 전처리하기
자연어 처리에는 다양한 말뭉치 데이터가 있는 데, 유명한 것으로 위키백과와 구글 뉴스같은 것이 있다. 파이썬의 대화모드(라고 하는군요)를 통해 간단한 텍스트를 전처리해보자.

In [1]:
text = 'You say goodbye and I say hello.'
text = text.lower()     # 소문자 변환
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [2]:
words = text.split(' ')     # 여기서는 공백으로 분리했지만 정규표현식으로 '(\W+)?, text)'로 분리하는 고급방식도 있다.
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [92]:
import re

txt = "RegExr was created by gskinner.com."
txt2 = "안녕하세요. 김ㅇㅇ입니다. 010-1234-5678입니다."
test1 = re.split("[\W+?]", txt2)
test1

['안녕하세요', '', '김ㅇㅇ입니다', '', '010', '1234', '5678입니다', '']

In [5]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)    # 0부터 순차적으로 들어간 갯수를 id로 사용
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [4]:
word_to_id, id_to_word

({'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6},
 {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'})

In [6]:
print(id_to_word[1])
print(word_to_id['hello'])

say
5


In [7]:
# 단어를 id로 구성해보자
import numpy

corpus = [word_to_id[w] for w in words]
corpus

[0, 1, 2, 3, 4, 1, 5, 6]

In [8]:
# 위의 과정을 함수로 만들어보자
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = numpy.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [10]:
text= 'I was born to love you'
corpus, word_to_id, id_to_word = preprocess(text)
corpus, word_to_id, id_to_word

(array([0, 1, 2, 3, 4, 5]),
 {'i': 0, 'was': 1, 'born': 2, 'to': 3, 'love': 4, 'you': 5},
 {0: 'i', 1: 'was', 2: 'born', 3: 'to', 4: 'love', 5: 'you'})

### 2.3.2 단어의 분산 표현
난 몰랐어 내 마음이 이리 다채로운지. 
색을 RGB로 표현할 수 있듯이(like vector), 단어도 벡터로 표현할 수 있을까? 벡터 표현은 '단어의 의미'를 파악할 수 있는 것을 말한다.  
'비색'이라고 하면 알 수 없지만 (R, G, B) = (170, 33, 22)라면 빨간계열의 색임을 알 수 있듯이 말이다.  
이를 자연어에서는 단어의 분산표현(distributional representation)이라고 한다.

### 2.3.3 분포 가설
자연어를 벡터로 만드는 것들은 간단한 아이디어에 뿌리를 두고 있다. 그 아이디어는 바로 '단어의 의미는 주변 단어에 의해 형성된다'는 분포 가설(distributional hypothesis)이다.  
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-3.png?raw=true)

### 2.3.4 동시발생 행렬
분포 가설에 기초해 단어를 벡터로 나타내는 방법은 **주변 단어를 세어보는 방법**이다.

In [11]:
import sys
sys.path.append('..')
import numpy as np

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)
print(id_to_word)       # 뭐야 아까 했던거임

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


옆에 등장하는 단어를 통해 빈도 수를 세어 표로 정리하면 아래와 같은 표가 나오는 데 , 이를 **동시발생 행렬(co-occurrence matrix)** 이라고 한다.  
이 표의 각 행은 해당 단어를 표현한 벡터가 된다. 
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-7.png?raw=true)
이걸 왜 손으로 그리죠

In [14]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0 ,1, 0, 0, 0]
], dtype=np.int32)
# 이걸 왜 수기로 써야하죠?! 생략합니다

In [45]:
# 당연히 자동화할 수 있습니다.
# corpus, vocab_size, window_size를 입력받아 행렬을 만드는 함수를 만들어봅니다.
def create_co_matrix(corpus: list, vocab_size: int, window_size=1):
    """Create co-occurrence matrix

    Args:
        corpus (list): 단어의 분산표현?
        vocab_size (int): 단어 사전 크기
        window_size (int, optional): 옆에 몇개까지 볼 것인지. Defaults to 1.

    Returns:
        np.array: 동시발생행렬
    """
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [37]:
text = "안녕하세요. 저는 김예신 입니다. 만나서 반갑습니다."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
co_matrix = create_co_matrix(corpus, vocab_size)
co_matrix


array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0]], dtype=int32)

In [38]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6, 1])

### 2.3.5 벡터 간 유사도
앞에서 동시발생 행렬을 활용해 단어를 벡터로 표현했다. 벡터 사이의 유사도는 어떻게 측정할 수 있는가?  
벡터 사이의 유사도를 측정하는 방법은 다양하다. 대표적으로 벡터의 내적이나 유클리드 거리 등을 꼽을 수 있다. 그 외에도 다양하지만 단어 벡터의유사도를 나타낼 때는 코사인 유사도를 자주 사용한다.
두 벡터 $x = {x_1, x_2, ..., x_n}$ 과 $y = {y_1, y_2, ..., y_n}$ 사이의 코사인 유사도는 아래와 같이 정의된다.
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/e%202-1.png?raw=true)

분자에는 벡터의 내적이, 분모에는 각 벡터의 크기를 나타내는 노름(norm)이 들어간다. 여기에서는 L2 normdmf rPtksgksek. (벡터의 각 원소를 제곱 해 더한 후 다시 제곱근을 구해 계산)  
이 식의 핵심은 **벡터를 정규화하고 내적을 구하는 것**이다.
> NOTE_ 코사인 유사도를 직관적으로 풀면 가리키는 방향이 얼마나 유사한가. 이다. 두 벡터의 방향이 완전히 일치하면 1, 완전히 반대면 -1, 두 벡터가 직교하면 0이 된다.

In [39]:
# 유사도를 구하는 함수를 만들어봅니다.
def cos_similarity(x, y):
    nx = x / np.sqrt(np.sum(x**2))
    ny = y / np.sqrt(np.sum(y**2))
    return np.dot(nx, ny)

이렇게 구현해도 가능하지만, 인수로 제로 벡터가 들어오면 분모가 0이 되어 계산이 불가능하다. 이를 방지하기 위해 분모에 아주 작은 값을 더해준다.

In [40]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps)
    ny = y / np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx, ny)

In [41]:
# 지금까지 만든 것으로 코사인 유사도를 구해봅니다.

vector_i = co_matrix[word_to_id['안녕하세요']]
vector_j = co_matrix[word_to_id['만나서']]

cos_similarity(vector_i, vector_j)      # 0. 70710로 비교적 유사한 단어임을 알 수 있다.

0.7071067758832467

### 2.3.6 유사 단어의 랭킹 표시
코사인 유사도를 구했으니, 유사단어의 순위를 나타내보자.  
어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력하는 함수를 만들어보자!
```python
most_similar(query, word_to_id, id_to_word, word_matrix, top=5)     # 뭐 이렇게 많아요. 단순화 하겠습니다. 내부로 함수를 다 집어넣어서요.
```

In [57]:
def most_similar(text, query, top=5):
    # 기본 전처리 및 동시발생행렬 생성
    corpus, word_to_id, id_to_word = preprocess(text)
    vocab_size = len(word_to_id)
    co_matrix = create_co_matrix(corpus, vocab_size)

    # 쿼리 확인
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('[query] ' + query)
    query_id = word_to_id[query]
    query_vec = co_matrix[query_id]

    # 코사인 유사도 계산
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(co_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순 정렬 및 출력
    for i in (-1 * similarity).argsort()[1:top+1]:
        print(f'{id_to_word[i]}: {similarity[i]}')

In [58]:
# text
most_similar(text, '안녕하세요')

[query] 안녕하세요
저는: 0.7071067758832467
입니다: 0.7071067758832467
만나서: 0.7071067758832467
반갑습니다: 0.7071067758832467
.: 0.0


In [59]:
# 책에서 하라고 한대로 따라하기
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # 쿼리 확인
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    similarity = np.zeros(len(word_to_id))
    for i in range(len(word_to_id)):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순 정렬 및 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(f'{id_to_word[i]}: {similarity[i]}')

        count += 1
        if count >= top:
            return

In [62]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)

[query] you
goodbye: 0.7071067758832467
i: 0.7071067758832467
hello: 0.7071067758832467
say: 0.0
and: 0.0


In [63]:
text = "안녕하세요. 저는 김예신 입니다. 만나서 반갑습니다."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('안녕하세요', word_to_id, id_to_word, C, top=5)

[query] 안녕하세요
저는: 0.7071067758832467
입니다: 0.7071067758832467
만나서: 0.7071067758832467
반갑습니다: 0.7071067758832467
.: 0.0


하지만 유사도가 쬐끔이상하다. 안녕하세요와 입니다가 유사도가 높다는 것은 우리와 직관과 거리가 멀다. 물론 지금은 말뭉치의 크기가 너무 작은 것이 원인입니다.  
지금까지 동시발생 행렬을 이용하면 단어를 벡터로 표현할 수 있다. 이것으로 통계 기반 기법의 '기본'을 마치겠다. 다음 절에서 지금의 방법을 개선해봐요~